<table>
    <tr>
        <td><img src="./img/Macc.png" width="auto"/></td>
        <td>
            <table><tr>
            <h1 style="color:blue;text-align:center">Lógica para Ciencias de la Computación</h1></td>
            </tr></table>   
        <td>&nbsp;</td>
        <td>
            <table><tr>
            <tp><p style="font-size:150%;text-align:center">Taller</p></tp>
            <tp><p style="font-size:150%;text-align:center">Representación de situaciones</p></tp>
            </tr></table>
        </td>
    </tr>
</table>

---

# Objetivo <a class="anchor" id="inicio"></a>

En clase hemos visto cómo representar situaciones, sus restricciones, y el objetivo que se busca resolver, por medio de fórmulas lógicas. El objetivo de este taller es explorar cómo implementar en Python la representación de situaciones.

# Secciones

1. [Problema de ejemplo.](#prob)
2. [Descriptores.](#des)
3. [Implementación de restricciones.](#imp)

# Problema de ejemplo <a class="anchor" id="prob"></a>

([Volver al inicio](#inicio))

Trabajaremos sobre el siguiente problema. Buscamos llenar todas las casillas en una tabla 2x2 con un número de 0 a 3, sin repetir. Por ejemplo:

![ejemplo](img/tabla.png)

Tenemos tres restricciones que debemos implementar para resolver el problema:

1. Un número sólo está en una casilla.
2. No hay más de un número en una casilla.
3. Debe haber por lo menos un número en una casilla.


# Descriptores <a class="anchor" id="des"></a>

([Volver al inicio](#inicio))

Debemos primero representar las letras proposicionales, las cuales cruzan la información de qué número está en qué casilla:

$OenCasilla_{x,y,n}$ es verdadera sii el número $n$ está en la casilla $(x,y)$

Para hacer la implementación en python, sugerimos usar la clase `Descriptor`, que se encuentra en la librería `Logica`.  Básicamente, lo que hace la codificación es representar un átomo como un solo caracter (es decir, una cadena de longitud 1).

Importamos la clase a partir de la librería:

In [3]:
from itertools import product
import numpy as np

class Formula :

    def __init__(self) :
        pass

    def __str__(self) :
        if type(self) == Letra:
            return self.letra
        elif type(self) == Negacion:
            return '-' + str(self.subf)
        elif type(self) == Binario:
            return "(" + str(self.left) + self.conectivo + str(self.right) + ")"

    def letras(self):
        if type(self) == Letra:
            return set(self.letra)
        elif type(self) == Negacion:
            return self.subf.letras()
        elif type(self) == Binario:
            return self.left.letras().union(self.right.letras())

    def subforms(self):
        if type(self) == Letra:
            return [str(self)]
        elif type(self) == Negacion:
            return list(set([str(self)] + self.subf.subforms()))
        elif type(self) == Binario:
            return list(set([str(self)] + self.left.subforms() + self.right.subforms()))

    def valor(self, I) :
        if type(self) == Letra:
            return I[self.letra]
        elif type(self) == Negacion:
            return not self.subf.valor(I)
        elif type(self) == Binario:
            if self.conectivo == 'Y':
                return self.left.valor(I) and self.right.valor(I)
            if self.conectivo == 'O':
                return self.left.valor(I) or self.right.valor(I)
            if self.conectivo == '>':
                return not self.left.valor(I) or self.right.valor(I)
            if self.conectivo == '=':
                return (self.left.valor(I) and self.right.valor(I)) or (not self.left.valor(I) and not self.right.valor(I))

class Letra(Formula) :
    def __init__ (self, letra:str) :
        self.letra = letra

class Negacion(Formula) :
    def __init__(self, subf:Formula) :
        self.subf = subf

class Binario(Formula) :
    def __init__(self, conectivo:str, left:Formula, right:Formula) :
        assert(conectivo in ['Y','O','>','='])
        self.conectivo = conectivo
        self.left = left
        self.right = right

def inorder_to_tree(cadena:str):
    conectivos = ['Y', 'O', '>', '=']
    if len(cadena) == 1:
        return Letra(cadena)
    elif cadena[0] == '-':
        return Negacion(inorder_to_tree(cadena[1:]))
    elif cadena[0] == "(":
        counter = 0 #Contador de parentesis
        for i in range(1, len(cadena)):
            if cadena[i] == "(":
                counter += 1
            elif cadena[i] == ")":
                counter -=1
            elif cadena[i] in conectivos and counter == 0:
                return Binario(cadena[i], inorder_to_tree(cadena[1:i]),inorder_to_tree(cadena[i + 1:-1]))
    else:
        raise Exception('¡Cadena inválida!')

class Descriptor :

    '''
    Codifica un descriptor de N argumentos mediante un solo caracter
    Input:  args_lista, lista con el total de opciones para cada
                     argumento del descriptor
            chrInit, entero que determina el comienzo de la codificación chr()
    Output: str de longitud 1
    '''

    def __init__ (self,args_lista,chrInit=256) :
        self.args_lista = args_lista
        assert(len(args_lista) > 0), "Debe haber por lo menos un argumento"
        self.chrInit = chrInit
        self.rango = [chrInit, chrInit + np.prod(self.args_lista)]

    def check_lista_valores(self,lista_valores) :
        for i, v in enumerate(lista_valores) :
            assert(v >= 0), "Valores deben ser no negativos"
            assert(v < self.args_lista[i]), f"Valor debe ser menor o igual a {self.args_lista[i]}"

    def codifica(self,lista_valores) :
        self.check_lista_valores(lista_valores)
        cod = lista_valores[0]
        n_columnas = 1
        for i in range(0, len(lista_valores) - 1) :
            n_columnas = n_columnas * self.args_lista[i]
            cod = n_columnas * lista_valores[i+1] + cod
        return cod

    def decodifica(self,n) :
        decods = []
        if len(self.args_lista) > 1:
            for i in range(0, len(self.args_lista) - 1) :
                n_columnas = np.prod(self.args_lista[:-(i+1)])
                decods.insert(0, int(n / n_columnas))
                n = n % n_columnas
        decods.insert(0, n % self.args_lista[0])
        return decods

    def P(self,lista_valores) :
        codigo = self.codifica(lista_valores)
        return chr(self.chrInit+codigo)

    def inv(self,codigo) :
        n = ord(codigo)-self.chrInit
        return self.decodifica(n)

def visualizar_formula(A,D):
    '''
    Visualiza una fórmula A (como string en notación inorder) usando el descriptor D
    '''
    vis = []
    for c in A:
        if c == '-':
            vis.append(' no ')
        elif c in ['(', ')']:
            vis.append(c)
        elif c in ['>', 'Y', 'O']:
            vis.append(' ' + c + ' ')
        elif c == '=':
            vis.append(' sii ')
        else:
            try:
                vis.append(D.escribir(c))
            except:
                raise("¡Caracter inválido!")
    return ''.join(vis)

Ahora creamos un descriptor de tres argumentos, dos para la casilla $(x,y)$ y uno para los números:

In [4]:
Nx = 2
Ny = 2
Nn = Nx * Ny
X = list(range(Nx))
Y = list(range(Ny))
numeros = list(range(Nn))
OenCasilla = Descriptor([Nx, Ny, Nn])

Métodos importantes del `Descriptor`:

* P(`lista_valores`): Codifica el cruce de información correspondiente a los valores en `lista_valores` mediante un caracter.
* inv(`caracter`): Decodifica `caracter` en una lista de valores. Esta lista depende del número de argumentos con que se haya inicializado el objeto descriptor. 

### Codificando en un solo caracter

Mediante el método P() podemos crear las codificaciones. A continuación presentaremos cada uno de los caracteres que codifican los cruces de información de que un número $n$ (con $n\in\{0, 1, 2, 3\}$) se encuentra en una casilla $(x,y)$ (donde $x,y\in\{0, 1\}$):

In [5]:
print("Cantidad de átomos OenCasilla:", OenCasilla.rango[1] - OenCasilla.rango[0])
print("Caracteres correspondientes a los átomos OenCasilla:\n")
for n in range(Nn):
    for x in range(Nx):
        for y in range(Ny):
            atomo = OenCasilla.P([x,y,n])
            print(f"Que el número {numeros[n]} está en la casilla ({X[x]},{Y[y]}) es el átomo {atomo}")
    print("")

Cantidad de átomos OenCasilla: 16
Caracteres correspondientes a los átomos OenCasilla:

Que el número 0 está en la casilla (0,0) es el átomo Ā
Que el número 0 está en la casilla (0,1) es el átomo Ă
Que el número 0 está en la casilla (1,0) es el átomo ā
Que el número 0 está en la casilla (1,1) es el átomo ă

Que el número 1 está en la casilla (0,0) es el átomo Ą
Que el número 1 está en la casilla (0,1) es el átomo Ć
Que el número 1 está en la casilla (1,0) es el átomo ą
Que el número 1 está en la casilla (1,1) es el átomo ć

Que el número 2 está en la casilla (0,0) es el átomo Ĉ
Que el número 2 está en la casilla (0,1) es el átomo Ċ
Que el número 2 está en la casilla (1,0) es el átomo ĉ
Que el número 2 está en la casilla (1,1) es el átomo ċ

Que el número 3 está en la casilla (0,0) es el átomo Č
Que el número 3 está en la casilla (0,1) es el átomo Ď
Que el número 3 está en la casilla (1,0) es el átomo č
Que el número 3 está en la casilla (1,1) es el átomo ď



### Decodificando el caracter

Ahora podemos incluir un método para visualizar más fácilmente la información que porta cada letra proposicional. Esto es, en nuestro ejemplo en cuestión, nuestro objeto `OenCasilla` representa el cruce de información de que un número se encuentra en un lugar. Entonces, al decodificar una letra proposicional, queremos que nos presente claramente esta información y no un caracter inpronunciable (aunque es precisamente este caracter el que usará la máquina). 

Para ello podemos usar el siguiente método `escribir`: 

In [6]:
def escribir(self, literal):
    if '-' in literal:
        atomo = literal[1:]
        neg = ' no'
    else:
        atomo = literal
        neg = ''
    x, y, n  = self.inv(atomo)
    return f"El número {numeros[n]}{neg} está en la casilla ({X[x]},{Y[y]})"
    
from types import MethodType

OenCasilla.escribir = MethodType(escribir, OenCasilla)

In [7]:
atomo = OenCasilla.P([0,1,2])
print(f"El caracter que codifica es {atomo}")
print("\nSu decodificación es:")
OenCasilla.escribir(atomo)

El caracter que codifica es Ċ

Su decodificación es:


'El número 2 está en la casilla (0,1)'

Este método también toma en cuenta si el literal es positivo o negativo:

In [8]:
OenCasilla.escribir('-' + atomo)

'El número 2 no está en la casilla (0,1)'

# Implementación de restricciones <a class="anchor" id="imp"></a>

([Volver al inicio](#inicio))

Ahora es necesario crear las reglas que limitarán los valores de verdad para las letras proposicionales. En nuestro problema de ejemplo tenemos tres restricciones:

1. Un número sólo está en una casilla.
2. No hay más de un número en una casilla.
3. Debe haber por lo menos un número en una casilla.

## Restricción 1

Comencemos por considerar la restricción 1, que dice que un número sólo está en una casilla:

$$\bigwedge_{x\in\{0,1\}}\bigwedge_{y\in\{0,1\}}\bigwedge_{n\in Numeros}\left(OenCasilla_{x,y,n}\to\neg\left(\bigvee_{(u,v)\neq (x,y)} OenCasilla_{u,v,n}\right)\right)$$

Saber cómo implementar toda esta fórmula de un solo tirón, con itorias y otorias anidadas, no es fácil. Desarrollaremos esta implementación siguiendo una serie de pasos, cada uno con mayor generalidad que el anterior.

### Paso 1

La fórmula para representar que si el 0 está en la casilla $(0,0)$, entonces no puede estar en ninguna otra casilla, es la siguiente:

$$OenCasilla_{0,0,0} \to \neg\left(\bigvee_{(u,v)\neq (0,0)} OenCasilla_{u,v,0}\right)$$

Esta fórmula se implementa en Python de la siguiente manera:

In [9]:
x = 0
y = 0
casilla = (x,y)
otras_casillas = [(i,j) for i in X for j in Y if (i,j) != (x,y)]
n = 0
formula1 = ''
inicial = True
for casilla in otras_casillas:
    u = casilla[0]
    v = casilla[1]
    if inicial:
        formula1 = OenCasilla.P([u,v,n])
        inicial = False
    else:
        formula1 = "(" + formula1 + "O" + OenCasilla.P([u,v,n]) + ")"

formula1 = "(" + OenCasilla.P([x,y,n]) + ">-" + formula1 + ")"
print(formula1)

(Ā>-((ĂOā)Oă))


Observe que la fórmula resultante es difícil de entender a simple vista, aunque un computador pueda trabajarla fácilmente. Para visualizarla de manera más comprensible y entender el significado del string generado por el código anterior, usaremos la función `visualizar_formula` que se encuentra en la librería `Logica`:

In [11]:

print(visualizar_formula(formula1, OenCasilla))

(El número 0 está en la casilla (0,0) >  no ((El número 0 está en la casilla (0,1) O El número 0 está en la casilla (1,0)) O El número 0 está en la casilla (1,1)))


### Paso 2

Para el siguiente paso, se debe replicar este procedimiento para los demás números. Es decir, se debe crear una regla que indique que, para cada número $n$, si $n$ ocupa la primera casilla, entonces $n$ no está en ninguna otra casilla. La fórmula de lógica proposicional que expresa esta restricción es la siguiente:

$$\bigwedge_{n\in Numeros}\left(OenCasilla_{0,0,n}\to\neg\left(\bigvee_{(u,v)\neq (0,0)} OenCasilla_{u,v,n}\right)\right)$$


**Ejercicio 1:** Implemente la fórmula anterior como un string de Python.

---

In [68]:
for f in numeros:
    x = 0
    y = 0
    casilla = (x,y)
    otras_casillas = [(i,j) for i in X for j in Y if (i,j) != (x,y)]
    n = numeros[f]
    formula1 = ''
    inicial = True
    for casilla in otras_casillas:
        u = casilla[0]
        v = casilla[1]
        if inicial:
            formula1 = OenCasilla.P([u,v,n])
            inicial = False
        else:
            formula1 = "(" + formula1 + "O" + OenCasilla.P([u,v,n]) + ")"
                
    formula1 = "(" + OenCasilla.P([x,y,n]) + ">-" + formula1 + ")"
    print(formula1)
    print(visualizar_formula(formula1, OenCasilla))
    

(Ā>-((ĂOā)Oă))
(El número 0 está en la casilla (0,0) >  no ((El número 0 está en la casilla (0,1) O El número 0 está en la casilla (1,0)) O El número 0 está en la casilla (1,1)))
(Ą>-((ĆOą)Oć))
(El número 1 está en la casilla (0,0) >  no ((El número 1 está en la casilla (0,1) O El número 1 está en la casilla (1,0)) O El número 1 está en la casilla (1,1)))
(Ĉ>-((ĊOĉ)Oċ))
(El número 2 está en la casilla (0,0) >  no ((El número 2 está en la casilla (0,1) O El número 2 está en la casilla (1,0)) O El número 2 está en la casilla (1,1)))
(Č>-((ĎOč)Oď))
(El número 3 está en la casilla (0,0) >  no ((El número 3 está en la casilla (0,1) O El número 3 está en la casilla (1,0)) O El número 3 está en la casilla (1,1)))


### Paso 3

Ahora que hemos creado la restricción para la casilla $(0,0)$, el siguiente paso consiste en replicar esta misma para todas las demás casillas. Con esto, tendremos que cada número podrá estar en a lo sumo una casilla. La fórmula lógica que expresa esta restricción es la siguiente:

$$\bigwedge_{x\in\{0,1\}}\bigwedge_{y\in\{0,1\}}\bigwedge_{n\in Numeros}\left(OenCasilla_{x,y,n}\to\neg\left(\bigvee_{(u,v)\neq (x,y)} OenCasilla_{u,v,n}\right)\right)$$


**Ejercicio 2:** Implemente la fórmula anterior como un string de Python.

---

In [16]:
for f in numeros:
    for y in Y:
        for x in X:

            casilla = (x,y)
            otras_casillas = [(i,j) for i in X for j in Y if (i,j) != (x,y)]
            n = numeros[f]
            formula1 = ''
            inicial = True
            for casilla in otras_casillas:
                u = casilla[0]
                v = casilla[1]
                if inicial:
                    formula1 = OenCasilla.P([u,v,n])
                    inicial = False
                else:
                    formula1 = "(" + formula1 + "O" + OenCasilla.P([u,v,n]) + ")"

            formula1 = "(" + OenCasilla.P([x,y,n]) + ">-" + formula1 + ")"
            print(formula1)
            print(" ")
            print(visualizar_formula(formula1, OenCasilla))
            print(" ")

(Ā>-((ĂOā)Oă))
 
(El número 0 está en la casilla (0,0) >  no ((El número 0 está en la casilla (0,1) O El número 0 está en la casilla (1,0)) O El número 0 está en la casilla (1,1)))
 
(ā>-((ĀOĂ)Oă))
 
(El número 0 está en la casilla (1,0) >  no ((El número 0 está en la casilla (0,0) O El número 0 está en la casilla (0,1)) O El número 0 está en la casilla (1,1)))
 
(Ă>-((ĀOā)Oă))
 
(El número 0 está en la casilla (0,1) >  no ((El número 0 está en la casilla (0,0) O El número 0 está en la casilla (1,0)) O El número 0 está en la casilla (1,1)))
 
(ă>-((ĀOĂ)Oā))
 
(El número 0 está en la casilla (1,1) >  no ((El número 0 está en la casilla (0,0) O El número 0 está en la casilla (0,1)) O El número 0 está en la casilla (1,0)))
 
(Ą>-((ĆOą)Oć))
 
(El número 1 está en la casilla (0,0) >  no ((El número 1 está en la casilla (0,1) O El número 1 está en la casilla (1,0)) O El número 1 está en la casilla (1,1)))
 
(ą>-((ĄOĆ)Oć))
 
(El número 1 está en la casilla (1,0) >  no ((El número 1 está en la

## Restricción 2

La segunda restricción es que no puede haber más de un número en cada casilla. Es decir, si el número $n$ se encuentra dentro de la casilla $(x,y)$, otro número $m\neq n$ no puede estar dentro de ella. La fórmula lógica que representa esta restricción es la siguiente:

$$\bigwedge_{x\in\{0,1\}}\bigwedge_{y\in\{0,1\}}\bigwedge_{n\in Numeros}\left(OenCasilla_{x,y,n}\to\neg\left(\bigvee_{m\neq n} OenCasilla_{x,y,m}\right)\right)$$


**Ejercicio 3:** Implemente la fórmula anterior como un string de Python.

In [79]:
for f in numeros:
    for y in Y:
        for x in X:

            casilla = (x,y,n)
            otras_casillas = [(i,j,m) for i in X for j in Y for n in numeros if m != n]
            n = numeros[f]
            formula1 = ''
            inicial = True
            for casilla in otras_casillas:
                u = casilla[0]
                v = casilla[1]
                for numeros in otros_numeros:
                    if inicial:
                        formula1 = OenCasilla.P([x,y,n])
                        inicial = False
                    else:
                        formula1 = "(" + formula1 + "O" + OenCasilla.P([u,v,n]) + ")"

                formula1 = "(" + OenCasilla.P([x,y,n]) + ">-" + formula1 + ")"
                print(formula1)
                print(" ")
                print(visualizar_formula(formula1, OenCasilla))
                print(" ")

NameError: name 'm' is not defined

---

## Restricción 3

La tercera restricción es que debe haber por lo menos un número en cada casilla. La fórmula lógica que representa esta restricción es la siguiente:

$$\bigwedge_{x\in\{0,1\}}\bigwedge_{y\in\{0,1\}}\left(\bigvee_{n\in Numeros}OenCasilla_{x,y,n}\right)$$


**Ejercicio 4:** Implemente la fórmula anterior como un string de Python.

---

En este taller usted aprendió a:

1) Empaquetar información mediante letras proposicionales.

2) Usar un objeto descriptor para cruzar información en Python.

3) Implementar las notaciones $\displaystyle\bigwedge_{x\in SET}$  y  $\displaystyle\bigvee_{x\in SET}$ dentro de Python.

4) Implementar restricciones como un string que contiene una fórmula en notación inorder.